# Loading pokemon data

In [1]:
import requests

url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
response = requests.get(url)
data = response.json()
pokemon_urls = [pokemon['url'] for pokemon in data['results']]

In [ ]:
counter = 0
pokemon_data = []
for url in pokemon_urls:
    response = requests.get(url)
    pokemon = response.json()
    base_stats = {stat['stat']['name']: stat['base_stat'] for stat in pokemon['stats']}
    pokemon_data.append({
        'name': pokemon['name'],
        'weight': pokemon['weight'],
        'hp': base_stats.get('hp', 0),
        'attack': base_stats.get('attack', 0),
        'defense': base_stats.get('defense', 0),
        'special_attack': base_stats.get('special-attack', 0),
        'special_defense': base_stats.get('special-defense', 0),
        'speed': base_stats.get('speed', 0),
        'type': [t['type']['name'] for t in pokemon['types']],
    })
    counter += 1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233


# Saving as dataframe

In [37]:
import pandas as pd 
all_pokemon_df = pd.DataFrame(pokemon_data)
all_pokemon_df.to_parquet('pokemons_stats.parquet')
all_pokemon_df

,name,weight,hp,attack,defense,special_attack,special_defense,speed,type,BST
0,bulbasaur,69,45,49,49,65,65,45,"[grass, poison]",318
1,ivysaur,130,60,62,63,80,80,60,"[grass, poison]",405
2,venusaur,1000,80,82,83,100,100,80,"[grass, poison]",525
3,charmander,85,39,52,43,60,50,65,[fire],309
4,charmeleon,190,58,64,58,80,65,80,[fire],405
...,...,...,...,...,...,...,...,...,...,...
1297,ogerpon-wellspring-mask,398,80,120,84,60,96,110,"[grass, water]",550
1298,ogerpon-hearthflame-mask,398,80,120,84,60,96,110,"[grass, fire]",550
1299,ogerpon-cornerstone-mask,398,80,120,84,60,96,110,"[grass, rock]",550
1300,terapagos-terastal,160,95,95,110,105,110,85,[normal],600


# Modeling the problem

In [33]:
import pulp

prob = pulp.LpProblem("Pokemon Team Optimization", pulp.LpMaximize)

x = pulp.LpVariable.dicts("x", range(len(pokemon_data)), cat='Binary')

prob += pulp.lpSum(
    (pokemon['hp'] + pokemon['attack'] + pokemon['defense'] + pokemon['special_attack'] + pokemon['special_defense'] + 
     pokemon['speed']) * x[i] for i, pokemon in enumerate(pokemon_data)
), "Total Combat Effectiveness"

# total weight constraint
max_weight_capacity = 1000
prob += pulp.lpSum(pokemon['weight'] * x[i] for i, pokemon in enumerate(pokemon_data)) <= max_weight_capacity, "Weight Capacity"

# total number of Pokemon constraint
prob += pulp.lpSum(x[i] for i in range(len(pokemon_data))) == 6, "Team Size"

prob.solve()

1

In [40]:
selected_pokemon = [pokemon_data[i]['name'] for i in range(len(pokemon_data)) if pulp.value(x[i]) == 1]
print("Selected Pokémon:", selected_pokemon)
selected_pokemon_data = [pokemon_data[i] for i in range(len(pokemon_data)) if pulp.value(x[i]) == 1]

Selected Pokémon: ['jirachi', 'meloetta-aria', 'pecharunt', 'mewtwo-mega-y', 'diancie-mega', 'eternatus-eternamax']


# Adding visualization

In [48]:
# Calculate BST for all Pokémon
import plotly
import plotly.graph_objects as go

for pokemon in pokemon_data:
    pokemon['BST'] = (pokemon['hp'] + pokemon['attack'] + pokemon['defense'] +
                                             pokemon['special_attack'] + pokemon['special_defense'] + 
                                             pokemon['speed'])

for pokemon in selected_pokemon_data:
    pokemon['total_combat_effectiveness'] = (pokemon['hp'] + pokemon['attack'] + pokemon['defense'] +
                                             pokemon['special_attack'] + pokemon['special_defense'] + 
                                             pokemon['speed'])
    
all_pokemon_df = pd.DataFrame(pokemon_data)
selected_df = pd.DataFrame(selected_pokemon_data)

# Create the scatter plot
fig = go.Figure()

# Add all Pokémon
fig.add_trace(go.Scatter(
    x=all_pokemon_df['weight'],
    y=all_pokemon_df['BST'],
    mode='markers',
    name='All Pokémon',
    marker=dict(size=10, color='blue', opacity=0.6),
    text=all_pokemon_df['name']
))

# Add selected Pokémon
fig.add_trace(go.Scatter(
    x=selected_df['weight'],
    y=selected_df['total_combat_effectiveness'],
    mode='markers',
    name='Selected Pokémon',
    marker=dict(size=12, color='red', opacity=0.9),
    text=selected_df['name']
))

# Add title and labels, resize
fig.update_layout(
    title="Pokémon Selection Optimization",
    xaxis_title="Weight (KGs)",
    yaxis_title="BST",
    legend_title="Legend",
    width=1000, 
    height=600,
    showlegend=True
)

# Show the plot
fig.show()